In [10]:
import pandas as pd
import numpy as np
import sys
import torch

In [25]:
with open("./MixSNIPS_clean/train.txt","r") as f:
    obj = f.readlines()

In [26]:
def processing(content):
    text = []
    slot = []
    intent = []
    
    text_tmp = ""
    slot_tmp = []
        
    for i in content:
        temp = i.strip("\n")
        if len(temp) == 0 :
            text.append(text_tmp.strip())
            slot.append(slot_tmp)
            text_tmp = ""
            slot_tmp = []
            
        line = temp.split()
        if len(line) == 1:
            intent.append(temp.split("#"))
        elif len(line) > 1:
            text_tmp += f"{line[0]} "
            slot_tmp.append(line[1])
            
    return text, slot, intent

In [27]:
text, slot, intent = processing(obj)

In [28]:
def multilabel2one_hot(labels, nums):
    res = [0.] * nums
    if len(labels) == 0:
        return res
    if isinstance(labels[0], list):
        for label in labels[0]:
            res[label] = 1.
        return res
    for label in labels:
        res[label] = 1.
    return res


def instance2onehot(func, num_intent, data):
    res = []
    for intents in func(data):
        res.append(multilabel2one_hot(intents, num_intent))
    return np.array(res)

In [29]:
# from collections import OrderedDict
# from ordered_set import OrderedSet

# class vocab_sheet(object):

#     def __init__(self, labels):
    
#         self.index2instance = OrderedSet()
#         self.instance2index = OrderedDict()
#         self.one_hot = []
    
#     def build_vocab(self):
#         pass

In [30]:
index2instance = list(set([j for i in intent for j in i]))
instance2index = {j:i for i,j in enumerate(index2instance)}

In [31]:
def one_hot_mapping(labels):
    one_hot = []
    for label in labels:
        tmp = [0.] * len(instance2index)
        for i in label:
            tmp[instance2index[i]] = 1
        one_hot.append(tmp)

    return torch.tensor(one_hot)

In [32]:
one_hot_mapping(intent)

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.]])

In [33]:
index2instance

['BookRestaurant',
 'SearchCreativeWork',
 'GetWeather',
 'PlayMusic',
 'AddToPlaylist',
 'SearchScreeningEvent',
 'RateBook']

In [34]:
len(instance2index)

7

In [35]:
# index2instance_train = index2instance

In [36]:
# index2instance_dev = index2instance

In [37]:
# index2instance_test = index2instance

In [38]:
# index2instance = set(index2instance_train +index2instance_dev +index2instance_test)

In [39]:
# index2instance

In [12]:
# f = open(f"./vocab.txt","w")
# f.writelines([line + "\n" for line in index2instance])
# f.close()

In [13]:
# f = open(f"./vocab.txt", "r")
# f.read().splitlines()

['GetWeather',
 'SearchCreativeWork',
 'PlayMusic',
 'AddToPlaylist',
 'RateBook',
 'SearchScreeningEvent',
 'BookRestaurant']

In [42]:
text

['play isham jones and swine not deserves four points',
 'play the last song by goldie and then can you get me the rakuen tsuihou: expelled from paradise television show',
 'find a soundtrack called the spanish jade',
 'rate rajinikanth: the definitive biography one out of 6 stars and then what s the movie schedule for b&b theatres',
 'put ramy ayach on latin pop rising and find the schedule for the kentuckians',
 'lets take me cherry and tracie to a noyack bar and then what are the showtimes for boy of mine',
 'i want to add kiara to my digster future hits playlist , give the book men and the city 3 out of 6 stars and show me the book f-1 grand prix part ii',
 'book a diner for 1 in green isle and then will it be warmer at 15 o clock in deepwater bonaire',
 'i need a reservation for a gastropub that serves a maple bacon donut',
 'list films in the neighborhood',
 'movie schedules at southern theatres',
 '4 stars for three to get deadly',
 'book a table for 10 people at dunbrody countr

In [46]:
one_hot = one_hot_mapping(intent)

In [60]:
uniq_oh = one_hot.unique(dim=0)

In [118]:
sent1 = []
sent2 = []
one_hot_col = []

for i in uniq_oh:
    indexes = torch.nonzero((one_hot == i).sum(dim=1) == one_hot.size(1)).squeeze(1)
    for j in range(indexes.size(0)):
        sent1.append(text[indexes[j]])
        if j == indexes.size(0)-1:
            sent2.append(text[indexes[0]])
        else:
            sent2.append(text[indexes[j+1]])
    one_hot_col.append(i.expand(indexes.size(0),7))


In [129]:
pd.DataFrame({"sent1": sent1 , "sent2" :sent2})

,sent1,sent2
0,4 stars for three to get deadly,rate this chronicle 0
1,rate this chronicle 0,rate this textbook 3 out 6
2,rate this textbook 3 out 6,i give the journal of a sad hermaphrodite a 0 ...
3,i give the journal of a sad hermaphrodite a 0 ...,rate current novel two stars
4,rate current novel two stars,i give why orwell matters a rating value of 2 ...
...,...,...
39771,"book a highly rated restaurant , what s the we...",book me a pub nine weeks from now for 4 people...
39772,book me a pub nine weeks from now for 4 people...,"reserve a smoking room at the restaurant , wil..."
39773,"reserve a smoking room at the restaurant , wil...","i d like a bar with hawaiian food in nd , what..."
39774,"i d like a bar with hawaiian food in nd , what...",need a table at a restaurant serving foie gras...


In [3]:
import torch

In [1]:
def accuracy_for_multi_label(pred, target, topk=(1,)):
    pred = torch.round(torch.sigmoid(pred))
    correct = (pred == target).all(dim=1).sum()
    res = correct / (target.size(0))
    return ((res,1),(res,2))


def cal_score(outputs, labels):
    corrects = 0
    totals = 0
    preds = 0
    acc = 0

    for i,logit in enumerate(outputs):

        log = torch.sigmoid(logit)

        correct = (labels[i][torch.where(log>0.5)[0]]).sum()

        total = len(torch.where(labels[i] == 1)[0])

        pred = len(torch.where(log>0.5)[0])

        corrects += correct

        totals += total

        preds += pred

        p = (torch.where(log>0.5)[0])

        r = (torch.where(labels[i]==1)[0])

        if len(p) == len(r) and (p == r).all():
            acc+=1

    return corrects, totals ,preds, acc


In [33]:
logits = torch.tensor([
    [1,1,1,1,1,0,0],
    [1,1,1,0,1,0,0],
    [1,1,1,1,0,1,0],
    [1,1,1,1,1,0,0],
    [1,1,1,1,0,1,0],
])

In [34]:
logits.size()

torch.Size([5, 7])

In [35]:
labels = torch.tensor([
    [1,1,1,1,0,0,0],
    [1,1,1,1,0,0,0],
    [1,1,1,1,0,0,0],
    [1,1,1,1,0,0,0],
    [1,1,1,1,0,0,0],
])

In [36]:
cal_score(logits, labels)

(tensor(19), 20, 24, 0)

In [37]:
accuracy_for_multi_label(logits, labels)

((tensor(0.), 1), (tensor(0.), 2))